In [1]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil

import pandas as pd
import pygeohash
import s3fs

import uuid
import math

In [2]:
endpoint_url = 'https://storage.budsc.midwest-datascience.com'

In [3]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

In [4]:
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
kv_dir = results_dir.joinpath('kv')
kv_dir.mkdir(parents=True, exist_ok=True)

In [5]:
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
            
    return records

In [6]:
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
            
    return flat_record

In [7]:
def create_flatten_dataset():
    records = read_jsonl_data()
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

In [8]:
df = create_flatten_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)
df['key_init'] = df['key'].astype(str).str[0]

In [9]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

7.a

In [10]:
def kv_key(row):
    for letter in partitions:
        if row['key_init'] in letter:
            if letter[0] == letter[1]:
                # return (print(letter[0]))
                part = letter[0]
            else:
                # return (print(f'{letter[0]}-{letter[1]}'))
                part = f'{letter[0]}-{letter[1]}'
        
            return part

df['kv_key'] = df.apply(lambda row: kv_key(row), axis=1)
# df

In [11]:
import pyarrow as pa
import pyarrow.parquet as pq 

table = pa.Table.from_pandas(df)

In [12]:
pq.write_to_dataset(
    table,
    root_path=f'{kv_dir}',
    partition_cols=['kv_key']
)

In [13]:
parquet_path = results_dir.joinpath('routes-flattened.parquet')
print(parquet_path)
# pq.write_table(table,f'{parquet_path}')

/home/jovyan/dsc650/dsc650/assignments/assignment07/results/routes-flattened.parquet


7.b

Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [14]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest().capitalize()

In [15]:
df[['key','key_init','kv_key']]

,key,key_init,kv_key
0,AERKZN2B,A,A
1,ASFKZN2B,A,A
2,ASFMRV2B,A,A
3,CEKKZN2B,C,C-D
4,CEKOVB2B,C,C-D
...,...,...,...
67658,WYAADLZL,W,W-X
67659,DMEFRUZM,D,C-D
67660,FRUDMEZM,F,E-F
67661,FRUOSSZM,F,E-F


In [16]:
df['hashed'] = df.apply(lambda row: hash_key(row['key']), axis=1)

In [17]:
df[['key','key_init','kv_key','hashed']]

,key,key_init,kv_key,hashed
0,AERKZN2B,A,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...
1,ASFKZN2B,A,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2,ASFMRV2B,A,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...
3,CEKKZN2B,C,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...
4,CEKOVB2B,C,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...
...,...,...,...,...
67658,WYAADLZL,W,W-X,F31527be84c36208c05cac57dfac8a46b48a87dda151f8...
67659,DMEFRUZM,D,C-D,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...
67660,FRUDMEZM,F,E-F,E976939986fbf947bb9318018cef717c0b34dff91e5e67...
67661,FRUOSSZM,F,E-F,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...


In [18]:
df['hash_key'] = df['hashed'].astype(str).str[0]
df[['key','key_init','kv_key','hashed','hash_key']]

,key,key_init,kv_key,hashed,hash_key
0,AERKZN2B,A,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6
1,ASFKZN2B,A,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9
2,ASFMRV2B,A,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1
3,CEKKZN2B,C,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3
4,CEKOVB2B,C,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1
...,...,...,...,...,...
67658,WYAADLZL,W,W-X,F31527be84c36208c05cac57dfac8a46b48a87dda151f8...,F
67659,DMEFRUZM,D,C-D,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...,8
67660,FRUDMEZM,F,E-F,E976939986fbf947bb9318018cef717c0b34dff91e5e67...,E
67661,FRUOSSZM,F,E-F,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...,8


In [19]:
hash_dir = results_dir.joinpath('hash')
hash_dir.mkdir(parents=True, exist_ok=True)

In [20]:
table = pa.Table.from_pandas(df)

pq.write_to_dataset(
    table,
    root_path=f'{hash_dir}',
    partition_cols=['hash_key']
)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   airline_airline_id      67663 non-null  int64  
 1   airline_name            67663 non-null  object 
 2   airline_alias           67663 non-null  object 
 3   airline_iata            67663 non-null  object 
 4   airline_icao            67663 non-null  object 
 5   airline_callsign        67663 non-null  object 
 6   airline_country         67663 non-null  object 
 7   airline_active          67663 non-null  bool   
 8   src_airport_airport_id  67180 non-null  float64
 9   src_airport_name        67180 non-null  object 
 10  src_airport_city        67180 non-null  object 
 11  src_airport_country     67180 non-null  object 
 12  src_airport_iata        67180 non-null  object 
 13  src_airport_icao        67180 non-null  object 
 14  src_airport_latitude    67180 non-null

In [33]:
df.src_airport_country.value_counts()

United States              13100
China                       8212
United Kingdom              2663
Spain                       2531
Germany                     2352
                           ...  
Tuvalu                         1
American Samoa                 1
Lesotho                        1
Falkland Islands               1
Cocos (Keeling) Islands        1
Name: src_airport_country, Length: 225, dtype: int64

In [34]:
#  total number of missing values in each column
df.isna().sum()

airline_airline_id          0
airline_name                0
airline_alias               0
airline_iata                0
airline_icao                0
airline_callsign            0
airline_country             0
airline_active              0
src_airport_airport_id    483
src_airport_name          483
src_airport_city          483
src_airport_country       483
src_airport_iata          483
src_airport_icao          483
src_airport_latitude      483
src_airport_longitude     483
src_airport_altitude      483
src_airport_timezone      483
src_airport_dst           483
src_airport_tz_id         483
src_airport_type          483
src_airport_source        483
dst_airport_airport_id    488
dst_airport_name          488
dst_airport_city          488
dst_airport_country       488
dst_airport_iata          488
dst_airport_icao          488
dst_airport_latitude      488
dst_airport_longitude     488
dst_airport_altitude      488
dst_airport_timezone      488
dst_airport_dst           488
dst_airpor

In [37]:
us_airports = df[df['src_airport_country'] == 'United States']
us_airports.src_airport_city.value_counts()

Atlanta              915
Chicago              697
New York             614
Los Angeles          492
Dallas-Fort Worth    469
                    ... 
Hydaburg               1
Adak Island            1
Kirksville             1
Twin Falls             1
Port Moller            1
Name: src_airport_city, Length: 530, dtype: int64

In [40]:
us_airports.groupby(['src_airport_city','src_airport_name']).count()

,,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_country,...,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,key_init,kv_key,hashed,hash_key
src_airport_city,src_airport_name,,,,,,,,,,,,,,,,,,,,,
Aberdeen,Aberdeen Regional Airport,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Abilene,Abilene Regional Airport,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Adak Island,Adak Airport,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Akhiok,Akhiok Airport,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Akiak,Akiak Airport,3,3,3,3,3,3,3,3,3,3,...,0,0,0,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wrangell,Wrangell Airport,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Yakima,Yakima Air Terminal McAllister Field,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Yakutat,Yakutat Airport,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [41]:
us_airports[['src_airport_name','src_airport_city','src_airport_country','src_airport_latitude','src_airport_longitude']]

,src_airport_name,src_airport_city,src_airport_country,src_airport_latitude,src_airport_longitude
170,Kodiak Airport,Kodiak,United States,57.750000,-152.494003
171,Kodiak Airport,Kodiak,United States,57.750000,-152.494003
172,Kodiak Airport,Kodiak,United States,57.750000,-152.494003
173,Kodiak Airport,Kodiak,United States,57.750000,-152.494003
174,Kodiak Airport,Kodiak,United States,57.750000,-152.494003
...,...,...,...,...,...
67565,Sheridan County Airport,Sheridan,United States,44.769199,-106.980003
67566,Show Low Regional Airport,Show Low,United States,34.265499,-110.005997
67567,Show Low Regional Airport,Show Low,United States,34.265499,-110.005997
67569,Visalia Municipal Airport,Visalia,United States,36.318699,-119.392998


In [ ]:
def airport_search(latitude, longitude):
    geohash = pygeohash.encode(latitude, longitude)
    dist_dict = {}
    for record in records:
        dist = pygeohash.geohash_approximate_distance(str(geohash), str(record.get('geohash')))
        dist_dict[dist] = record.get('src_airport')
        
        print(list(sorted(dist_dict.items()))[0][1]['name'])
        pass
    
airport_search(41.1499988, -95.91779)